In [12]:
import sys
sys.path.append("../scripts")

In [13]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import optimizers
from keras import losses
from keras import metrics

import os
import logging
from PIL import Image
from matplotlib import cm

In [3]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [4]:
import tad_data
import tad_model
import tad_loss
import tad_metric

In [5]:
width = 2048
n_joints = 25
label_count = 50
stride = 2

skeleton_dir = "../data/skeleton.7z"
label_dir = "../data/label.7z"

learning_rate = .01
input_len = 800
load_batch_size = 100
batch_size = 8
epochs = 1000
loss_weights = [1., 1., 1.]

conf_threshold_base = 0.1
iou_threshold = 0.6

In [ ]:
model = tad_model.get_model(width, n_joints, label_count)

In [7]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=learning_rate),
    loss=[tad_loss.HeatMapCrossEntropy(), losses.Huber(), losses.Huber()],
    loss_weights=loss_weights
)

In [6]:
model = keras.models.load_model("../models/model_800.keras")

I0000 00:00:1727039893.723190    1108 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727039893.883896    1108 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727039893.883942    1108 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727039893.887248    1108 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727039893.887289    1108 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [7]:
input_dataset, target_dataset = tad_data.get_dataset_from_compressed(skeleton_dir, label_dir, width, n_joints, stride, label_count, input_len, load_batch_size)

Reading from skeleton/0002-L.txt ... skeleton/0035-L.txt
Reading from skeleton/0035-M.txt ... skeleton/0068-R.txt
Reading from skeleton/0069-L.txt ... skeleton/0102-L.txt
Reading from skeleton/0102-M.txt ... skeleton/0135-M.txt
Reading from skeleton/0135-R.txt ... skeleton/0168-R.txt
Reading from skeleton/0169-L.txt ... skeleton/0202-L.txt
Reading from skeleton/0202-M.txt ... skeleton/0235-M.txt
Reading from skeleton/0235-R.txt ... skeleton/0272-R.txt


In [ ]:
model.evaluate(input_dataset, target_dataset)

In [8]:
# model.optimizer.learning_rate /= 10
model.optimizer.learning_rate.numpy()

9.999999e-06

In [17]:
model.fit(input_dataset,
          target_dataset,
          batch_size=batch_size, 
          epochs=epochs,)
          # callbacks=[tad_loss.LearningSchedule()],)
                     # tad_metric.MeanAveragePrecision(input_dataset, target_dataset, stride, conf_threshold_base, iou_threshold)])

Epoch 1/1000
100/100 ━━━━━━━━━━━━━━━━━━━━ 5s 47ms/step - conv2d_18_loss: 3.3794e-07 - conv2d_20_loss: 5.3580e-05 - conv2d_22_loss: 0.0012 - loss: 0.0012
Epoch 2/1000
100/100 ━━━━━━━━━━━━━━━━━━━━ 4s 44ms/step - conv2d_18_loss: 3.3195e-07 - conv2d_20_loss: 5.2904e-05 - conv2d_22_loss: 0.0011 - loss: 0.0012
Epoch 3/1000
100/100 ━━━━━━━━━━━━━━━━━━━━ 4s 44ms/step - conv2d_18_loss: 3.3086e-07 - conv2d_20_loss: 5.5096e-05 - conv2d_22_loss: 0.0012 - loss: 0.0013
Epoch 4/1000
100/100 ━━━━━━━━━━━━━━━━━━━━ 4s 44ms/step - conv2d_18_loss: 3.5198e-07 - conv2d_20_loss: 5.3987e-05 - conv2d_22_loss: 0.0012 - loss: 0.0013
Epoch 5/1000
100/100 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - conv2d_18_loss: 3.4145e-07 - conv2d_20_loss: 5.0490e-05 - conv2d_22_loss: 0.0011 - loss: 0.0011
Epoch 6/1000
100/100 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - conv2d_18_loss: 3.3712e-07 - conv2d_20_loss: 5.6262e-05 - conv2d_22_loss: 0.0011 - loss: 0.0012
Epoch 7/1000
100/100 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - conv2d_18_loss: 3.4228e-0

In [21]:
model.save("../models/model_800.keras")

In [19]:
out_dataset = model.predict((input_dataset[0], input_dataset[1]))

25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step


In [35]:
import importlib

importlib.reload(tad_metric)

<module 'tad_metric' from '/home/rinzler/projects/untrimmed_video/notebooks/../scripts/tad_metric.py'>

In [20]:
tad_metric.mAP(out_dataset, target_dataset, stride=stride, conf_threshold_base=conf_threshold_base, iou_threshold=iou_threshold)

0.9278059194344033

In [17]:
idx = 777

out = model.predict((input_dataset[0][idx:idx+1], input_dataset[1][idx:idx+1]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step


In [19]:
img_list = list()
img_pad_big = np.full((100, 1024), 0.1)
img_pad_small = np.full((10, 1024), 0.05)

img_list.append(img_pad_big)
for channel in range(label_count):
    img_out = np.transpose(out[0][0,:,:,channel])
    img_out = np.broadcast_to(img_out, (200, 1024))
    
    img_target = np.transpose(target_dataset[0][idx][:,:,channel])
    img_target = np.broadcast_to(img_target, (200, 1024))

    img_list.append(img_out)
    img_list.append(img_pad_small)
    img_list.append(img_target)
    img_list.append(img_pad_big)
    
img = np.concatenate(img_list)
Image.fromarray(cm.bone(img, bytes=True)).save("../images/display.png")

In [18]:
peak_threshold = 0.1

img_list = list()
img_pad_big = np.full((100, 2048), 0.1)
img_pad_small = np.full((10, 2048), 0.05)

img_all_list = list()
img_all_out = np.zeros((1, 2048))
img_all_target = np.zeros((1, 2048))

img_list.append(img_pad_big)
for channel in range(label_count):
    img_out = np.zeros((1, 2048))
    for i, o in enumerate(out[0][0,:,:,channel]):
        if o >= peak_threshold:
            c = (i + out[1][0,i,0,0]) * stride
            w = np.exp(out[2][0,i,0,0]) * stride

            ts = max(0, int(c - w/2))
            te = min(int(c + w/2), 2048 - 1)
            print(f"Out:\t{channel+1}\t{ts}\t{te}")
            for t in range(ts, te+1):
                img_out[0,t] = 1.;
                img_all_out[0,t] += (channel + 1) / label_count;
                img_all_out[0,t] = min(1., img_all_out[0,t])
    
    img_out = np.broadcast_to(img_out, (200, 2048))

    img_target = np.zeros((1, 2048))
    for i, o in enumerate(target_dataset[0][idx,:,:,channel]):
        if o == 1.:
            c = (i + target_dataset[1][idx,i,0,0]) * stride
            w = np.exp(target_dataset[2][idx,i,0,0]) * stride
            
            ts = max(0, int(c - w/2))
            te = min(int(c + w/2), 2048 - 1)
            print(f"Target:\t{channel+1}\t{ts}\t{te}")
            for t in range(ts, te+1):
                img_target[0,t] = 1.;
                img_all_target[0,t] += (channel + 1) / label_count;
                img_all_target[0,t] = min(1., img_all_target[0,t])
    
    img_target = np.broadcast_to(img_target, (200, 2048))

    img_list.append(img_out)
    img_list.append(img_pad_small)
    img_list.append(img_target)
    img_list.append(img_pad_big)
    
img = np.concatenate(img_list)
Image.fromarray(cm.bone(img, bytes=True)).save("../images/display.png")

img_all_out = np.broadcast_to(img_all_out, (200, 2048))
img_all_target = np.broadcast_to(img_all_target, (200, 2048))
img_all = np.concatenate([img_pad_big, img_all_out, img_pad_small, img_all_target, img_pad_big])
Image.fromarray(cm.bone(img_all, bytes=True)).save("../images/display_all.png")

Out:	1	1732	1761
Target:	1	1730	1763
Out:	2	113	147
Target:	2	111	150
Out:	3	601	612
Target:	3	600	613
Out:	4	740	760
Out:	4	1405	1448
Out:	4	1868	1900
Target:	4	1866	1903
Out:	5	1578	1605
Target:	5	1576	1607
Out:	6	243	266
Target:	6	242	267
Out:	13	180	208
Target:	13	178	211
Out:	17	37	60
Out:	17	1941	1962
Target:	17	35	62
Target:	17	1940	1963
Out:	22	663	713
Target:	22	657	720
Out:	23	1073	1139
Target:	23	1067	1146
Out:	28	436	467
Target:	28	434	469
Out:	29	1178	1209
Out:	29	1301	1338
Target:	29	1175	1212
Out:	31	1669	1695
Out:	31	1988	2024
Target:	31	1984	2029
Out:	32	740	760
Target:	32	739	762
Out:	33	1256	1268
Target:	33	1255	1270
Out:	34	1374	1382
Target:	34	1374	1383
Out:	35	510	529
Target:	35	508	531
Out:	37	921	967
Target:	37	918	971
Out:	38	1005	1026
Target:	38	1003	1028
Out:	40	1504	1541
Target:	40	1500	1545
Out:	43	1301	1338
Out:	43	1868	1900
Target:	43	1300	1339
Out:	45	1792	1828
Target:	45	1790	1831
Out:	47	601	612
Out:	47	1504	1541
Out:	47	1669	1695
Target:	47	1667	1698


In [15]:
# conf_threshold_base = 0.2

boxes_gt = list()
boxes_pred = list()

for i, (out_o, target_o) in enumerate(zip(out[0][0,:,0,:], target_dataset[0][idx,:,0,:])):
    for channel, o in enumerate(out_o):
        if o >= conf_threshold_base:
            c = (i + out[1][0,i,0,0]) * stride
            w = np.exp(out[2][0,i,0,0]) * stride
    
            ts = max(0, int(c - w/2))
            te = min(int(c + w/2), 2048 - 1)
            boxes_pred.append([ts, te, o])
            print(f"Out:\t{channel+1}\t{ts}\t{te}\t{o}")

    for channel, o in enumerate(target_o):
        if o == 1.:
            c = (i + target_dataset[1][idx,i,0,0]) * stride
            w = np.exp(target_dataset[2][idx,i,0,0]) * stride
            
            ts = max(0, int(c - w/2))
            te = min(int(c + w/2), 2048 - 1)
            boxes_gt.append([ts, te, o])
            print(f"Target:\t{channel+1}\t{ts}\t{te}\t{o}")
    
boxes_gt = np.array(boxes_gt)
boxes_pred = np.array(boxes_pred)

Out:	6	37	78	0.10064373910427094
Out:	50	37	78	0.9625813961029053
Target:	50	32	83	1.0
Out:	6	114	149	0.9431610107421875
Target:	6	110	153	1.0
Out:	32	195	225	0.9999319314956665
Target:	32	191	228	1.0
Out:	4	271	298	0.8904527425765991
Out:	39	271	298	0.1192912608385086
Target:	4	268	301	1.0
Out:	40	400	441	0.9887658953666687
Target:	40	394	447	1.0
Out:	9	448	467	0.9305402040481567
Target:	9	446	469	1.0
Out:	41	614	641	0.9076396226882935
Target:	41	611	644	1.0
Out:	40	703	766	0.14045092463493347
Out:	42	703	766	0.9165587425231934
Target:	42	694	775	1.0
Out:	23	784	849	0.11721836775541306
Out:	31	784	849	0.8633208274841309
Out:	40	784	849	0.1108846440911293
Target:	31	774	859	1.0
Out:	15	890	893	0.7543655037879944
Target:	15	856	927	1.0
Out:	13	953	990	0.9550780653953552
Out:	28	953	990	0.10660184919834137
Target:	13	948	995	1.0
Out:	38	1030	1032	0.16785579919815063
Out:	45	1045	1108	0.9383442401885986
Target:	45	1036	1117	1.0
Out:	43	1143	1208	0.9676403999328613
Target:	43	1133	1218	1.0

In [40]:
class StopExecution(Exception):
    def _render_traceback_(self):
        return []

In [27]:
# iou_threshold = 0.5

if boxes_gt.size == 0:
    raise StopExecution

if boxes_pred.size == 0:
    raise StopExecution

ious = tad_metric.get_ious(boxes_gt, boxes_pred)
matches_gt, matches_pred = tad_metric.get_matches(ious, iou_threshold)
labels, predictions = tad_metric.get_lp(boxes_gt, boxes_pred, matches_gt, matches_pred)

print(ious)
print()

print(matches_gt)
print(matches_pred)
print()

print(labels)
print(predictions)
print()

tad_metric.average_precision(labels, predictions)

[[0.80392157 0.80392157 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.        ]
 [0.         0.         0.81395349 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.        ]
 [0.         0.         0.         0.81081081 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.        ]
 [0.         0.         0.         0.         0.81818182 0

0.8940445908968282

In [16]:
def metrics(labels, predictions, conf_threshold):
    tp = 0
    fp = 0
    fn = 0
    
    for label, prediction in zip(labels, predictions):
        if label == 1.:
            if prediction > conf_threshold:
                tp += 1
            else:
                fn += 1
        else:
            if prediction > conf_threshold:
                fp += 1
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else float('nan')
    recall = tp / (tp + fn) if (tp + fn) > 0 else float('nan')
    
    print(tp, fp, fn)
    print(precision)
    print(recall)

In [134]:
t = target_dataset[0][i:i+1]
# t = out[0]

channel = 8

opt = np.get_printoptions()
np.set_printoptions(threshold=np.inf)

print(np.argmax(t[0,:,0,channel]))
print(t[0,438,0,channel])
print(t[0,:,0,channel])

np.set_printoptions(**opt)

438
1.0
[0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 